# ORTHOGONAL POLYNOMIAL DENSITY ESTIMATION
## Preliminaries
### Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sbn
import pandas as pd
from uuid import uuid4

from lpde.geometry import WidthOf, Window, PointAt, BoundingBox, Mapper
from lpde.estimate import DensityEstimate
from lpde.estimate.helpers import Event, Degree, Action

### Notebook settings

In [2]:
%matplotlib notebook
sbn.set_style('white')

legend_params = {'loc': 'best',
                 'frameon': True, 
                 'fancybox': True, 
                 'framealpha': 0.7}

In [7]:
legendre_width = WidthOf(1.8)

center = PointAt(0, 0)
window = Window(1.8, 1.8)
bounds = BoundingBox(center, window)

mapper = Mapper(bounds, legendre_width)

degree = Degree(20, 20)
density = DensityEstimate(degree, mapper)

action = Action.ADD
point = PointAt(0.5, 0.5)
event = Event(uuid4(), action, point)

In [8]:
def multi_norm():
    x, y = np.random.multivariate_normal((0,0), ((0.1,0), (0,0.1)))
    if (-0.9 <= x <= 0.9) and (-0.9 <= y <= 0.9):
        return x, y
    else:
        return multi_norm()

def multi_uniform():
    return np.random.uniform(low=-0.9, high=0.9, size=2)

def new_event():
    location = multi_uniform()
    point = PointAt(*location)
    return Event(uuid4(), Action(1), point)

def random_event():
    event_type = np.random.randint(low=-1, high=2)
    if event_type == 1:
        location = multi_uniform()
        point = PointAt(*location)
        return Event(uuid4(), Action(1), point)
    elif event_type == 0:
        location = multi_uniform()
        point = PointAt(*location)
        column = density._phi.sample(1, axis=1).columns.values[0]
        return Event(column, Action(0), point)
    column = density._phi.sample(1, axis=1).columns.values[0]
    return Event(column, Action(-1))

In [9]:
%%time
for i in range(1000):
    density.update_with(new_event())

/home/georg/Documents/Python/LPDE/lpde/estimate/estimator.py:88: RuntimeWarning: overflow encountered in double_scalars
  c[0] * (c[1:].dot(c[1:]) - 1.0)
/home/georg/Documents/Python/LPDE/lpde/estimate/estimator.py:87: RuntimeWarning: overflow encountered in square
  return -log(square(c[1:].dot(self.__phi_ijn))).sum() + \
/home/georg/Documents/Python/LPDE/lpde/estimate/estimator.py:93: RuntimeWarning: overflow encountered in multiply
  c[1:].dot(self.__phi_ijn)).sum(axis=1) + 2.0*c[0]*c[1:]


CPU times: user 9min 31s, sys: 11.2 s, total: 9min 42s
Wall time: 1min 2s


In [10]:
density._number_of_fallbacks

13

In [11]:
%%time
for i in range(1000):
    density.update_with(random_event())

CPU times: user 14min 1s, sys: 16.4 s, total: 14min 17s
Wall time: 1min 11s


In [ ]:
71/1000

In [12]:
%%time
for i in range(1000):
    density.at(new_event().location)

CPU times: user 604 ms, sys: 16 ms, total: 620 ms
Wall time: 580 ms


__Timings__

64 ms per additive update with uniform distribution

71 ms per random update with uniform distribution

0.6 ms per evaluation at point

In [ ]:
x_grid = np.linspace(-0.9, 0.90, 50)
y_grid = np.linspace(-0.9, 0.90, 50)
x_grid, y_grid = np.meshgrid(x_grid, y_grid)

p_hat = density._on(x_grid, y_grid)

fig, ax = plt.subplots()
ax.set(xlabel=r'$x$', ylabel=r'$y$')
contour = ax.contourf(x_grid, y_grid, p_hat, 9, cmap='inferno')
cbar = plt.colorbar(contour, ax=ax)
fig.tight_layout()